<a href="https://colab.research.google.com/github/RemyLpr/defi_ia/blob/main/model/Model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import des modules et des données

In [1]:
import pandas as pd
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFECV
import warnings
warnings.filterwarnings('ignore')

In [159]:
df = pd.read_csv("train_set.csv") # données d'entraînement
df_eval = pd.read_csv("test_set.csv") # données de test du site
df_hotel = pd.read_csv("features_hotels.csv") # données sur les hôtels
df_clust = pd.read_csv("hotel_price_clust.csv") # clusters sur les prix des hôtels à J44
df_stock_j44_clust = pd.read_csv("stock_j44_clust.csv") # clusters sur les stocks des hôtels à J44
df_stagn = pd.read_csv("golden.csv")

# Preview des données

In [ ]:
df

,hotel_id,price,stock,city,date,language,mobile,avatar_id,avatar_name,order_requests
0,768,286,13,madrid,11,portuguese,0,155051,8HOAWS,1
1,126,96,8,madrid,11,portuguese,0,155051,8HOAWS,1
2,378,124,0,madrid,11,portuguese,0,155051,8HOAWS,1
3,351,91,4,madrid,11,portuguese,0,155051,8HOAWS,1
4,747,172,15,madrid,11,portuguese,0,155051,8HOAWS,1
...,...,...,...,...,...,...,...,...,...,...
69273,118,101,3,vilnius,4,german,1,177245,M4JPKK,656
69274,954,135,0,vilnius,4,german,1,177245,M4JPKK,656
69275,589,61,0,vilnius,4,german,1,177245,M4JPKK,656
69276,432,157,4,vilnius,4,german,1,177245,M4JPKK,656


In [ ]:
df_eval

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock
0,0,1,vilnius,21,romanian,0,1,161,46
1,1,1,vilnius,21,romanian,0,1,187,32
2,2,1,vilnius,21,romanian,0,1,279,12
3,3,1,vilnius,21,romanian,0,1,395,10
4,4,1,vilnius,21,romanian,0,1,488,42
...,...,...,...,...,...,...,...,...,...
6639,6639,843,rome,5,irish,0,794,987,1
6640,6640,844,vienna,1,irish,1,794,26,1
6641,6641,844,vienna,1,irish,1,794,263,0
6642,6642,844,vienna,1,irish,1,794,456,0


# Préparation des données

Tout d'abord on homogénéise les colonnes (variables d'entrée des datasets de train et de test). On fait aussi la jointure avec le dataframe contenant les features des hôtels.

In [160]:
df.drop("avatar_name", axis = 1, inplace = True)
df_eval.drop("index", axis = 1, inplace = True)
df_hotel.drop("city", axis = 1, inplace = True) # on doit la supprimer immédiatement pour ne pas créer de doublon
df = pd.merge(df, df_hotel, on = "hotel_id", how = "left") # on merge avec le csv features_hotels
df_eval = pd.merge(df_eval, df_hotel, on = "hotel_id", how = "left")
df = pd.merge(df, df_clust, on = "hotel_id", how = "left") # on merge avec le csv hotel_price_clust
df_eval = pd.merge(df_eval, df_clust, on = "hotel_id", how = "left")
df = pd.merge(df, df_stock_j44_clust, on = "hotel_id", how = "left") # on merge avec le csv stock_j44_clust
df_eval = pd.merge(df_eval, df_stock_j44_clust, on = "hotel_id", how = "left")
mobile_list_1, mobile_list_2 = [], []

Le merge avec la feature stagne, qui varie selon la ville ET le jour, est un peu différent et nécessite un traitement à part. Aussi, un comportement étrange a été remarqué pour la ville Madrid au jour 2, pour certains hôtels la valeur de la variable mobile a une influence sur le prix, contrairement à habituellement. On va donc détecter quels sont ces hôtels et les stocker dans des listes pour modifier leur prix après.

In [164]:
# ATTENTION : cellule à réexécuter après création des mobile_list
stagn_list = []
for index, row in df.iterrows():
  city, hotel, date, mobile = row["city"], row["hotel_id"], row["date"], row["mobile"]
  if (date==2) & (mobile==1) & (hotel in mobile_list_2): # cas particulier repéré où la valeur de mobile a une influence pour la date 2
    stagn_list.append(round(0.92*df_stagn.iloc[hotel][date]))
  else:
    if (date==1) & (mobile==1) & (hotel in mobile_list_1): # pareil pour la date 1
      stagn_list.append(round(0.92*df_stagn.iloc[hotel][date]))
    else:
      stagn_list.append(df_stagn.iloc[hotel][date])
df["stagne"] = stagn_list
mobile_list_1, mobile_list_2 = [], [] # on vide les listes pour éviter des problèmes lors de potentielles réexécutions

In [162]:
stagn_list = []
for index, row in df_eval.iterrows():
  hotel, date = row["hotel_id"], row["date"]
  stagn_list.append(df_stagn.iloc[hotel][date])
df_eval["stagne"] = stagn_list

In [163]:
row_list = []
a = df[df["language"] == "hungarian"]
for index, row in a.iterrows():
  if abs(row["price"] - row["stagne"]) > 1:
    row_list.append(row)
df_errors = pd.DataFrame(data=row_list)
df_errors_1, df_errors_2 = df_errors[df_errors["date"] == 1], df_errors[df_errors["date"] == 2]
mobile_list_1, mobile_list_2 = df_errors_1["hotel_id"].tolist(), df_errors_2["hotel_id"].tolist()

In [200]:
df_greek = df[df["language"] == "spanish"]
df_hung = df[df["language"] == "hungarian"]

In [201]:
coeff_list = []
for index, row in df_greek.iterrows():
  hotel, date = row["hotel_id"], row["date"]
  df_match = df_hung[(df_hung["hotel_id"] == hotel) & (df_hung["date"] == date)]
  for index2, row2 in df_match.iterrows():
    coeff_list.append(row["price"]/row2["price"])

In [202]:
coeff_list

[1.0833333333333333,
 1.0761904761904761,
 1.0792079207920793,
 1.0872093023255813,
 1.0843373493975903,
 1.0909090909090908,
 1.0909090909090908,
 1.0830449826989619,
 1.083969465648855,
 1.0808080808080809,
 1.083916083916084,
 1.0818181818181818,
 1.0833333333333333,
 1.0851063829787233,
 1.0816326530612246,
 1.0807453416149069,
 1.086021505376344,
 1.0836653386454183,
 1.083743842364532,
 1.0776699029126213,
 1.0759493670886076,
 1.0761904761904761,
 1.0776699029126213,
 1.0838323353293413,
 1.0836363636363637,
 1.0827586206896551,
 1.084033613445378,
 1.0787878787878789,
 1.0833333333333333,
 1.0869565217391304,
 1.089430894308943,
 1.0862068965517242,
 1.0909090909090908,
 1.0851063829787233,
 1.0909090909090908,
 1.0827586206896551,
 1.083743842364532,
 1.0921052631578947,
 1.0817610062893082,
 1.0845070422535212,
 1.0941176470588236,
 1.0887573964497042,
 1.0869565217391304,
 1.0830324909747293,
 1.0816326530612246,
 1.0851063829787233,
 1.0909090909090908,
 1.0855855855855856,

In [199]:
coeff_list = np.array(coeff_list)
coeff_list.mean()

1.0604473509925607

In [185]:
df_hung.sort_values(by="date")

,hotel_id,price,stock,city,date,language,mobile,avatar_id,order_requests,group,brand,parking,pool,children_policy,cluster_hotel,stock_j44_clust,stagne
50443,890,76,0,vienna,4,hungarian,1,156969,476,Yin Yang,8 Premium,1,0,0,1,1,76.0
100703,890,76,0,vienna,9,hungarian,0,203567,945,Yin Yang,8 Premium,1,0,0,1,1,76.0
79108,890,76,0,vienna,10,hungarian,0,177475,750,Yin Yang,8 Premium,1,0,0,1,1,76.0
106740,890,70,45,vienna,41,hungarian,1,203622,1000,Yin Yang,8 Premium,1,0,0,1,1,70.0
132163,890,70,45,vienna,41,hungarian,1,203858,1236,Yin Yang,8 Premium,1,0,0,1,1,70.0


Pour l'instant on va supprimer les colonnes avatar_id, hotel_id et order_requests pour se faciliter la tâche. De nouvelles variables pourraient être générées à partir de celles-ci mais pour l'instant notre but est juste de faire tourner un premier modèle.

In [138]:
df.drop(["avatar_id", "hotel_id", "order_requests"], axis = 1, inplace = True)
df_eval.drop(["avatar_id", "hotel_id", "order_requests"], axis = 1, inplace= True)

Ensuite on trie les colonnes par ordre alphabétique pour une visualisation/comparaison plus claire de celles-ci.

In [139]:
df = df.reindex(sorted(df.columns), axis=1)
df_eval = df_eval.reindex(sorted(df_eval.columns), axis=1)

On doit encoder les variables catégorielles "city" et "language". Pour effectuer cette tâche nous allons utliser la méthode get_dummies de pandas car les modalités de ces variables n'ont pas de lien entre elles.

In [140]:
# génération des dummies pour les variables city, language, brand et group
cities = pd.get_dummies(df["city"], prefix_sep = "_", prefix = "city")
languages =  pd.get_dummies(df["language"], prefix_sep = "_", prefix = "language")
brands = pd.get_dummies(df["brand"], prefix_sep = "_", prefix = "brand")
groups = pd.get_dummies(df["group"], prefix_sep = "_", prefix = "group")
df = pd.concat([df, cities, languages, brands, groups], axis = 1) # on ajoute les colonnes générées au df
df.drop(["city", "language", "brand", "group"], axis = 1, inplace = True) # on supprime les colonnes d'origine

On effectue le même processus pour le dataset d'évaluation.

In [141]:
cities = pd.get_dummies(df_eval["city"], prefix_sep = "_", prefix = "city")
languages =  pd.get_dummies(df_eval["language"], prefix_sep = "_", prefix = "language")
brands = pd.get_dummies(df_eval["brand"], prefix_sep = "_", prefix = "brand")
groups = pd.get_dummies(df_eval["group"], prefix_sep = "_", prefix = "group")
df_eval = pd.concat([df_eval, cities, languages, brands, groups], axis = 1)
df_eval.drop(["city", "language", "brand", "group"], axis = 1, inplace = True)

## Train test split

Maintenant on sépare la variable de sortie (price) des variables d'entrée.

In [142]:
X = df.drop("price", axis = 1)
y = df["price"]

Enfin, on split nos données en entraînement/test (répartition 80/20).

In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Scaling des données

Partie non obligatoire : scaler les données pour permettre une meilleure prédiction du modèle. N'est pas nécessaire pour tous les modèles.

In [ ]:
scaler = StandardScaler()
X[['cluster_hotel','date', 'stock', 'stock_j44_clust']] = scaler.fit_transform(df[['cluster_hotel','date', 'stock', 'stock_j44_clust']]) 
df_eval[['cluster_hotel','date', 'stock', 'stock_j44_clust']] = scaler.fit_transform(df_eval[['cluster_hotel','date', 'stock', 'stock_j44_clust']])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Entraînement et test d'un modèle

Ici je vaiis utiliser un modèle de type MLPRegressor [https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor](https://).
Nous allons déclarer le modèle puis l'entraîner et le tester.

In [155]:
# import du régresseur
from sklearn.neural_network import MLPRegressor
# définition du régresseur utilisé et entraînement de celui-ci
reg = MLPRegressor(hidden_layer_sizes = (256, 256, 256), verbose = True, early_stopping = True, tol = 0.05, n_iter_no_change=1, validation_fraction=0.1).fit(X_train, y_train)

Iteration 1, loss = 104.94022182
Validation score: 0.998447


In [156]:
y_pred = reg.predict(X_test)
rmse = mean_squared_error(y_true = y_test, y_pred = y_pred, squared = False)
print('rmse : ', rmse)

rmse :  3.1829956792531617


# Export des résultats et évaluation sur Kaggle

On doit ensuite utiliser le dataset de test fourni par sur le Kaggle en renvoyer le fichier au format demandé.

In [157]:
eval = reg.predict(df_eval)
submission = pd.DataFrame(data = eval.tolist(), columns = ["price"])
submission.to_csv("MLP_first_iteration.csv", index_label='index') # tout ça pour créer la colonne Unnamed et la transformer en index (nécessaire pour l'évaluation par le site)